In [46]:
""" Import zone """

import html
import datetime

import numpy as np
import pandas as pd
import seaborn as sns

import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer; nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer

import spacy; spacy_nlp = spacy.load("en_core_web_sm", disable=['tagger', 'parser', 'ner', 'entity_linker', 'entity_ruler', 'textcat', 'textcat_multilabel', 'morphologizer', 'attribute_ruler'])

from comodo.comodo import *

import support
register_consts(support.comodo_config)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tommaso\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Done → COMODO.CONSTANTS in register_consts() → Comodo's costants correctly configured


In [8]:
step = 'clean'
trdf = DF.set_datetime_index(DF.rcsv(step+'/train_df.csv', sep='\t'), 'date')

In [17]:
all_pos_tag = UTILS.pkl_load('pkl/all_pos_tag')

In [18]:
""" Stopwords, Lemmatization and Filtering"""

lemmatizer = spacy_nlp.get_pipe("lemmatizer")
# lemmatizer = WordNetLemmatizer()
# stemmer = SnowballStemmer('english')

all_sentences_NJVR_lemmatized = []

for iR,review in enumerate(all_pos_tag):
    entry_review = []
    for iS,sentence in enumerate(review):
        entry_sentence = []
        
        # NLTK
        # [entry_sentence.append(lemmatizer.lemmatize(word[0]) if word[1][0]!='V' else stemmer.stem(word[0])) for word in sentence if word[1] in OBJECTS._POS_TAG['TO_KEEP'] and word[1] not in OBJECTS._STOPWORDS]
        
        # SPACY
        [entry_sentence.append(word[0]) for word in sentence if word[1] in OBJECTS._POS_TAG['TO_KEEP'] and word[1] not in OBJECTS._STOPWORDS]
        
        entry_review.append(entry_sentence)
    all_sentences_NJVR_lemmatized.append(entry_review)

In [19]:
""" Back to Dataset """

trdf['prep_review'] = trdf.apply(lambda r: '|'.join([' '.join(sentence) for sentence in all_sentences_NJVR_lemmatized[r['order']]]), axis = 1)
trdf.head(3)

,drug,condition,review,rating,date,useful,order,prep_review
date,,,,,,,,
2008-02-24,Chlorpheniramine / pseudoephedrine,Allergic Rhinitis,i when to a medical clinic with flu like sympt...,1.0,2008-02-24,0,0,medical clinic flu symptoms Dr. duty prescribe...
2008-02-24,Oxybutynin,Not Listed / Othe,improved my problem dramatically. i never expe...,7.0,2008-02-24,22,1,Improved problem dramatically|never experience...
2008-02-24,Xenical,Obesity,xenical really helped me but some of the bowel...,7.0,2008-02-24,50,2,Xenical really helped bowel problems had made ...


In [45]:
spacy_nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x23650c13280>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x23650c138e0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x23650976c80>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x23650ac0e40>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x23650ac1440>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x23650976e40>)]

In [42]:
trdf['norm_review'] = trdf['prep_review'].apply(lambda review: '|'.join([' '.join([word.lemma_ for word in spacy_nlp(sentence)]) for sentence in review.split('|')]))

KeyboardInterrupt: 